In [1]:

#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
# import pandas.stats.moments as st
import statsmodels.api as sm # import statsmodels 
from pandas import ExcelWriter
import matplotlib.pyplot as pyplot
import scipy.stats as st
import os
import quandl as qd
from collections import defaultdict
import seaborn as sns
%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Importing data from Quandl

btc = qd.get("BITFINEX/BTCUSD", authtoken="-ZxrLoLy3vKgLtLraaMn")
btc.columns = ['High','Low','Mid','Close','Bid','Ask','Volume']

In [3]:
def ichimoku_cloud(df,w9,w26,w52,chikou26):
    #crypto settings are 20, 60, 120, 30 see Josh video for confirmation
    #https://www.youtube.com/watch?v=5x0r-qcGoQQ&t=1s
    
    
    # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    df['period9_high'] = pd.Series.rolling(df['High'], w9).max()
    df['period9_low'] = pd.Series.rolling(df['Low'], w9).min()
    df['tenkan_sen'] = (df['period9_high'] + df['period9_low']) / 2
    
    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    df['period26_high'] = pd.Series.rolling(df['High'], w26).max()
    df['period26_low'] = pd.Series.rolling(df['Low'], w26).min()
    df['kijun_sen'] = (df['period26_high'] + df['period26_low']) / 2
    
    df['period52_high'] = pd.Series.rolling(df['High'], w52).max()
    df['period52_low'] = pd.Series.rolling(df['Low'], w52).min()
    df['senkou_span_a'] = (df['tenkan_sen'] + df['kijun_sen']) / 2
    df['senkou_span_b'] = (df['period52_high'] + df['period52_low'] ) /2 
    
    df ['chikou_span'] = df['Close'].shift(chikou26) 
    return df


In [4]:
ichimoku_cloud(btc,20,60,120,30) #crypto settings
#ichimoku_cloud(btc,9,26,52,26) #default settings 
btc = btc.dropna()

In [5]:
btc.iloc[0]

High               492.610000
Low                457.010000
Mid                478.815000
Close              478.200000
Bid                478.390000
Ask                479.240000
Volume           26077.595025
period9_high       605.720000
period9_low        443.000000
tenkan_sen         524.360000
period26_high      665.000000
period26_low       443.000000
kijun_sen          554.000000
period52_high      680.000000
period52_low       419.400000
senkou_span_a      539.180000
senkou_span_b      549.700000
chikou_span        616.120000
Name: 2014-08-19 00:00:00, dtype: float64

In [6]:
btc.iloc[1]

High               521.89000
Low                464.00000
Mid                510.74500
Close              510.50000
Bid                510.55000
Ask                510.94000
Volume           29808.26561
period9_high       605.72000
period9_low        443.00000
tenkan_sen         524.36000
period26_high      665.00000
period26_low       443.00000
kijun_sen          554.00000
period52_high      680.00000
period52_low       419.40000
senkou_span_a      539.18000
senkou_span_b      549.70000
chikou_span        622.97000
Name: 2014-08-20 00:00:00, dtype: float64

In [8]:

def returns_df(df):
    df['tk position'] = None
    df['tk senkou b position'] = None
    df['tk senkou a position'] = None
    for row in range(len(df)):
        if df['tenkan_sen'].iloc[row] > df['kijun_sen'].iloc[row]:
            df['tk position'].iloc[row] = 1
        else:
            df['tk position'].iloc[row] = -1

    for row in range(len(df)):
        if df['Close'].iloc[row] > df['senkou_span_b'].iloc[row]:
            df['tk senkou b position'].iloc[row] = 1
        else:
            df['tk senkou b position'].iloc[row] = -1

    for row in range(len(df)):
        if df['Close'].iloc[row] > df['senkou_span_a'].iloc[row]:
            df['tk senkou b position'].iloc[row] = 1
        else:
            df['tk senkou b position'].iloc[row] = -1
            
    df['tk position'].fillna(method = 'ffill', inplace = True)
    df['tk senkou b position'].fillna(method = 'ffill', inplace = True)
    df['market return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['strategy return tk'] = df['tk position'] * df['market return']
    df['strategy return senkou b'] = df['tk senkou b position'] * df['market return']
    df['strategy return senkou a'] = df['tk senkou a position'] * df['market return']
    
            
    return df

In [14]:
returns_df(btc)
btc.dropna(inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
def risk_measures(df):
    pass

In [15]:
#btc['strategy return tk'].cumsum().plot(grid = True, figsize = (20,5), color = 'g')
btc['strategy return senkou b'].cumsum().plot(grid = True, figsize = (20,5), color = 'b')
btc['strategy return senkou a'].cumsum().plot(grid = True, figsize = (20,5), color = 'r')
btc['market return'].cumsum().plot(grid = True, figsize = (20,5), color = 'm')

TypeError: Empty 'DataFrame': no numeric data to plot

In [ ]:
btc['market return'].cumsum().tail(5)